In [1]:
import napari
from napari_assistant._gui._Assistant import Assistant
from skimage.io import imread
from napari_assistant._categories import CATEGORIES

filename = 'C:/Users/ryans/OneDrive/Desktop/blobs.tif'
blobs = imread(filename)
blobs_modified = imread('C:/Users/ryans/OneDrive/Desktop/blobs_messed_up.tiff') 

viewer = napari.Viewer()
viewer.add_image(blobs_modified, name = 'not blobs')
viewer.add_image(blobs, name = 'blobs')

C:\Users\ryans\Anaconda3\envs\np_workflows_v1\lib\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'blobs' at 0x2c90d2e7580>

In [2]:
from napari_workflows._io_yaml_v1 import load_workflow

workflow = load_workflow(filename='test_only_blur.yaml')

In [3]:
from napari_assistant._workflow_io_utility import make_flexible_gui, wf_steps_with_root_as_input, set_choices, get_source_keywords_and_sources,category_kwargs,kwargs_of_wf_step

# find all workflow steps with functions which have root images as an input
root_functions = wf_steps_with_root_as_input(workflow)
layer_names = [lay.name for lay in viewer.layers]

# iterate over root function workflow steps
for wf_step_name in root_functions:
    # get the fuction from the workflow step and change its signature to
    # the values specified by the workflow
    func = workflow._tasks[wf_step_name][0]

    # create the widget based on the adjusted function and turn off autocall
    # for functions with more than 1 input image
    sources = workflow.sources_of(wf_step_name)
    if len(sources) > 1:
        widget = make_flexible_gui(func, 
                                   viewer,
                                   workflow,
                                   wf_step_name,
                                   autocall= False)
    else:
        widget = make_flexible_gui(func, 
                                   viewer, 
                                   workflow,
                                   wf_step_name)

    # determine if all input images are in layer names
    sources_present = True
    for source in sources:
        if source not in layer_names:
            sources_present = False

    # if all input images are present in the layers we can preselct them
    if sources_present:
        viewer.window.add_dock_widget(widget, name= wf_step_name[10:])
        set_choices(workflow= workflow,
                    wf_step_name= wf_step_name,
                    viewer= viewer,
                    widget= widget)
    
    # if the input images aren't present we will leave the dropdowns
    # open to chose the right image - needs fixing
    
    else:
        '''
            # make a tooltip which tells the user to select the input image
            # specified by the workflow
            key_source_list = get_source_keywords_and_sources(workflow,
                                                              wf_step_name)
            for key, source in key_source_list:
                widget[key].tooltip = f'Select {source} or equivalent'
        '''
        # add the final widget to the napari viewer
        viewer.window.add_dock_widget(widget, name = wf_step_name[10:] + '<b> - SELECT INPUT</b>')
    
    # setting the right parameters
    cat_kwargs = category_kwargs(
        func,
        kwargs_of_wf_step(workflow,wf_step_name),
    )
    for k,v in cat_kwargs.items():
        widget[k].value = v
    
    # calling the widget with the correct input images
    widget()

2022-04-12 16:52:43.327 | INFO     | napari_assistant._gui._category_widget:call_op:129 - gaussian_blur (clesperanto)(..., 1.0, 1.0, 0.0)
2022-04-12 16:52:43.356 | DEBUG    | napari_assistant._gui._category_widget:_show_result:231 - creating new layer for id: 3062376607312
2022-04-12 16:52:43.454 | INFO     | napari_assistant._gui._category_widget:call_op:129 - gaussian_blur (clesperanto)(..., 1.0, 1.0, 0.0)
2022-04-12 16:52:43.462 | DEBUG    | napari_assistant._gui._category_widget:_show_result:225 - updating existing layer: Result of gaussian_blur (clesperanto), with id: 3062376607312
2022-04-12 16:52:43.481 | INFO     | napari_assistant._gui._category_widget:call_op:129 - gaussian_blur (clesperanto)(..., 4.0, 1.0, 0.0)
2022-04-12 16:52:43.492 | DEBUG    | napari_assistant._gui._category_widget:_show_result:225 - updating existing layer: Result of gaussian_blur (clesperanto), with id: 3062376607312
2022-04-12 16:52:43.510 | INFO     | napari_assistant._gui._category_widget:call_op:12

set workflow step: Result of gaussian_blur (clesperanto)
     args: ['not blobs']
   kwargs: {'destination': None, 'sigma_x': 1.0, 'sigma_y': 1.0, 'sigma_z': 0.0}
signature: (source: Union[numpy.ndarray, pyclesperanto_prototype._tier0._pycl.OCLArray, pyopencl._cl.Image, pyclesperanto_prototype._tier0._pycl._OCLImage], destination: Union[numpy.ndarray, pyclesperanto_prototype._tier0._pycl.OCLArray, pyopencl._cl.Image, pyclesperanto_prototype._tier0._pycl._OCLImage] = None, sigma_x: float = 0, sigma_y: float = 0, sigma_z: float = 0) -> Union[numpy.ndarray, pyclesperanto_prototype._tier0._pycl.OCLArray, pyopencl._cl.Image, pyclesperanto_prototype._tier0._pycl._OCLImage]
set workflow step: Result of gaussian_blur (clesperanto)
     args: ['blobs']
   kwargs: {'destination': None, 'sigma_x': 1.0, 'sigma_y': 1.0, 'sigma_z': 0.0}
signature: (source: Union[numpy.ndarray, pyclesperanto_prototype._tier0._pycl.OCLArray, pyopencl._cl.Image, pyclesperanto_prototype._tier0._pycl._OCLImage], destinat

In [ ]:
widget

In [ ]:
widget.asdict()

In [ ]:
numeric = widget['x']
numeric.value = 3
numeric

In [ ]:
numeric.value = 1

In [ ]:
from napari_assistant._gui._category_widget import (
    separate_argnames_by_type, 
    num_positional_args, 
    category_args_bool,
    category_args_numeric,
    category_args_text,
)
from inspect import signature
def get_numeric_str_bool_param_dictionary(function,kwargs):
    cle_function = function
    nargs = num_positional_args(cle_function)
    category_kwargs = {}
    new_sig = signature(cle_function)
    # get the names of positional parameters in the new operation
    param_names, numeric_param_names, bool_param_names, str_param_names = separate_argnames_by_type(
        new_sig.parameters.items())

    # go through all parameters and collect their values in an args-array
    num_count = 0
    str_count = 0
    bool_count = 0
    for key in param_names: 
        if key in numeric_param_names:
            category_kwargs[category_args_numeric[num_count]] = kwargs[key]
            num_count = num_count + 1
        elif key in bool_param_names:
            category_kwargs[category_args_bool[bool_count]] = kwargs[key]
            bool_count = bool_count + 1
        elif key in str_param_names:
            category_kwargs[category_args_text[str_count]] = kwargs[key]
            str_count = str_count + 1
        
    return category_kwargs

def kwargs_of_wf_step(workflow,wf_step_name):
    func     = workflow._tasks[wf_step_name][0]
    arg_vals = workflow._tasks[wf_step_name][1:]

    # getting the keywords corresponding to the values
    keyword_list = list(signature(func).parameters.keys())

    # creating the kwargs dict
    kw_dict = {}
    for kw, val in zip(keyword_list, arg_vals):
        kw_dict[kw] = val
        
    return kw_dict

kwargs = kwargs_of_wf_step(workflow,'Result of gaussian_blur (clesperanto)')
print(f'kwargs: {kwargs}')
category_kwargs  =  get_numeric_str_bool_param_dictionary(func,kwargs)
print(f'category_kwargs: {category_kwargs}')

In [ ]:
for k,v in category_kwargs.items():
    widget[k].value = v